<a href="https://colab.research.google.com/github/uzeziogho/ETLDemo/blob/main/ETLDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!git clone https://github.com/uzeziogho/ETLDemo.git
%cd ETLDemo

#!pip install petl
#!pip install pymssql

import os
import sys
import petl
import pymssql
import configparser
import requests
import datetime
import json
import decimal


#get data from configuration file
config = configparser.ConfigParser()
try:
    config.read('ETLDemo.ini')
except Exception as e:
    print('could not read configuration file:'+ str(e))
    sys.exit()

#read settings from configuration file
startDate = config['CONFIG']['startDate']
url = config['CONFIG']['URL']
destServer = config['CONFIG']['server']
destDatabase = config['CONFIG']['database']
print(url)

Cloning into 'ETLDemo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 15.19 KiB | 15.19 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/ETLDemo/ETLDemo/ETLDemo/ETLDemo/ETLDemo/ETLDemo
https://www.bankofcanada.ca/valet/observations/FXUSDCAD/json?start_date=


In [18]:
#request data from url
try:
    BOCResponse = requests.get(url+startDate)
except Exception as e:
    print('could not get data from url:'+ str(e))
    sys.exit()

#initialise list of lists for data storage
BOCDates = []
BOCRates = []

#check response status ans process BOC JSON object
if (BOCResponse.status_code == 200):
    BOCRaw = json.loads(BOCResponse.text)

    #extract observation data into column arrays
    for row in BOCRaw['observations']:
      BOCDates.append(datetime.datetime.strptime(row['d'],'%Y-%m-%d'))
      BOCRates.append(decimal.Decimal(row['FXUSDCAD']['v']))

    # create petl table from column arrays and rename the columns
    exchangeRates = petl.fromcolumns([BOCDates,BOCRates],header=['date','rate'])

    #load expense document
    try:
        expenses = petl.io.xlsx.fromxlsx('Expenses.xlsx',sheet='Github')
    except Exception as e:
        print('could not open expenses file:'+ str(e))
        sys.exit()

        #join table
        expenses = petl.outerjoin(exchangeRates,expenses,lkey='date')

        #fill down missing values
        expenses = petl.filldown(expenses,'rate')

        #remove dates with no expenses
        expenses = petl.select(expenses,lambda rec: rec.USD != None)

        #add CDN column
        expenses = petl.addfield(expenses,'CAD',lambda rec: rec.USD * rec.rate)

        #calculate net expenses
        expenses = petl.addfield(expenses,'net',lambda rec: rec.USD - rec.rate * rec.CAD)


        #store results
#else:
   # print('Error connecting to server')